In [ ]:
import pandas as pd
# import pymysql
import sys
import os

In [ ]:
# # OP.GG Database에 접근하기 위한 연결 객체 생성
# con = pymysql.connect(
#     user = os.environ['LOL_KR_ID'],
#     passwd = os.environ['LOL_KR_PW'],
#     host = os.environ['LOL_KR_HOST'],
#     db = 'lol',
#     charset = 'utf8'
# )
# cursor = con.cursor(pymysql.cursors.DictCursor)

In [ ]:
# # 데이터 구조 파악을 위한 sample data fetch
# # SQL
# cursor.execute('''
# SELECT STRAIGHT_JOIN *
# FROM opGame o FORCE INDEX (ix_createDate),
# p_opGameStats p FORCE INDEX (`PRIMARY`)
# WHERE o.gameId = p.gameId
# AND o.createDate >= '2021-10-20'
# AND p.createDate >= '2021-10-20'
# ORDER BY o.createDate DESC
# LIMIT 3000
# ''')
# sample = cursor.fetchall()

# # DataFrame으로 변환
# sample = pd.DataFrame(sample)

# # 저장
# sample.to_csv("sample.csv", mode='w')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# 미리 뽑아둔 CSV 파일에서 가져오기
sample = pd.read_csv('sample.csv', index_col=0)

In [ ]:
# 데이터 확인
sample

In [ ]:
# Column 수가 많아서 print했을 때 다 보이게 옵션 설정
pd.set_option('display.max_columns', None)

In [ ]:
# columns 확인
print(sample.columns)

# 값이 정상적인지 보기 위해 describe로 요약 정보 출력
sample.describe(include='all',datetime_is_numeric=True)

In [ ]:
# 필요한 값만 선택
sample = sample[['gameId','subType','championId','result','position']]

In [ ]:
# 최종적으로 만들어진 sample dataframe
sample

In [ ]:
# Reset sample data
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
import pandas as pd
import numpy as np
# import pymysql
import sys
import os
from datetime import date, timedelta
from scipy.stats import norm
import requests

In [ ]:
# # OP.GG Database에 접근하기 위한 연결 객체 생성
# con = pymysql.connect(
#     user = os.environ['LOL_KR_ID'],
#     passwd = os.environ['LOL_KR_PW'],
#     host = os.environ['LOL_KR_HOST'],
#     db = 'lol',
#     charset = 'utf8'
# )
# cursor = con.cursor(pymysql.cursors.DictCursor)

In [ ]:
# # 패치 날짜 데이터 불러오기
# cursor.execute('''
# SELECT version, date
# FROM lolVersionHistory
# ''')
# patchDate = cursor.fetchall()

# patchDate = pd.DataFrame(patchDate)

# # version => season, patch로 가공, 세부 버전(핫픽스, 밸런싱 X 패치 등) 제거
# patchDate['season'] = pd.to_numeric(patchDate['version'].str.split('.').str[0])
# patchDate['patch'] = pd.to_numeric(patchDate['version'].str.split('.').str[1])
# patchDate = patchDate.groupby(['season','patch'],as_index=False).min('date').drop('version', axis = 1).sort_values(['season','patch'])

In [ ]:
# last_patch_start = patchDate.iloc[-2].date
# last_patch_end = patchDate.iloc[-1].date

In [ ]:
# bot_count = pd.DataFrame()
# spt_count = pd.DataFrame()
# duo_count = pd.DataFrame()

# for date in pd.date_range(last_patch_start,last_patch_end):
    
#     if date.strftime("%Y-%m-%d") == last_patch_start.strftime("%Y-%m-%d"):
#         date_start = last_patch_start.strftime("%Y-%m-%d %H:%M:%S")
#     else:
#         date_start = date.strftime("%Y-%m-%d 00:00:00")
        
#     if date.strftime("%Y-%m-%d") == last_patch_end.strftime("%Y-%m-%d"):
#         date_end = last_patch_end.strftime("%Y-%m-%d %H:%M:%S")
#     else:
#         date_end = (date+timedelta(1)).strftime("%Y-%m-%d 00:00:00")
        
#     # BOT data fetch
#     # SQL
#     cursor.execute(f'''
#     SELECT STRAIGHT_JOIN o.gameId, tierRank, championId, result
#     FROM opGame o FORCE INDEX (ix_createDate),
#     p_opGameStats p FORCE INDEX (`PRIMARY`)
#     WHERE o.gameId = p.gameId
#     AND o.createDate >= '{date_start}'
#     AND o.createDate < '{date_end}'
#     AND p.createDate >= '{date_start}'
#     AND p.createDate < '{date_end}'
#     AND position ='A'
#     AND result != 3
#     AND subType = 420
#     ''')
#     botstats = cursor.fetchall()
#     botstats = pd.DataFrame(botstats)
#     botstats['tierRank'] = botstats['tierRank'].str[0]
#     botstats = botstats[botstats.tierRank.isin(['P','D','M','R','C'])]
    
#     # SPT data fetch
#     # SQL
#     cursor.execute(f'''
#     SELECT STRAIGHT_JOIN o.gameId, tierRank, championId, result
#     FROM opGame o FORCE INDEX (ix_createDate),
#     p_opGameStats p FORCE INDEX (`PRIMARY`)
#     WHERE o.gameId = p.gameId
#     AND o.createDate >= '{date_start}'
#     AND o.createDate < '{date_end}'
#     AND p.createDate >= '{date_start}'
#     AND p.createDate < '{date_end}'
#     AND position ='S'
#     AND result != 3
#     AND subType = 420
#     ''')
#     sptstats = cursor.fetchall()
#     sptstats = pd.DataFrame(sptstats)
#     sptstats['tierRank'] = sptstats['tierRank'].str[0]
#     sptstats = sptstats[sptstats.tierRank.isin(['P','D','M','R','C'])]
    
#     # bot count
#     bot_count_new = botstats.groupby(['championId'],as_index=False).\
#     apply(lambda x: pd.Series({'win_count':sum(x['result'] == 'WIN'),
#                                'count':len(x)}))
    
#     # spt count
#     spt_count_new = sptstats.groupby(['championId'],as_index=False).\
#     apply(lambda x: pd.Series({'win_count':sum(x['result'] == 'WIN'),
#                                'count':len(x)}))
    
#     # duo count
#     duo_count_new = botstats.merge(sptstats,
#                                how = 'inner',
#                                on = ['gameId','result']).\
#     groupby(['championId_x','championId_y'],
#             as_index=False).\
#     apply(lambda x: pd.Series({'win_count':sum(x['result'] == 'WIN'),
#                                'count':len(x)}))
    
#     # concatenate
#     bot_count = pd.concat([bot_count, bot_count_new])
#     spt_count = pd.concat([spt_count, spt_count_new])
#     duo_count = pd.concat([duo_count, duo_count_new])

In [ ]:
# bot_count = bot_count.groupby(['championId'],as_index=False).\
# apply(lambda x: pd.Series({'WR':sum(x['win_count'])/sum(x['count']),
#                            'count':sum(x['count'])}))
# spt_count = spt_count.groupby(['championId'],as_index=False).\
# apply(lambda x: pd.Series({'WR':sum(x['win_count'])/sum(x['count']),
#                            'count':sum(x['count'])}))
# duo_count = duo_count.groupby(['championId_x','championId_y'],as_index=False).\
# apply(lambda x: pd.Series({'WR':sum(x['win_count'])/sum(x['count']),
#                            'count':sum(x['count'])}))

In [ ]:
# # 저장
# bot_count.to_csv("bot_count.csv", mode='w')
# spt_count.to_csv("spt_count.csv", mode='w')
# duo_count.to_csv("duo_count.csv", mode='w')

In [ ]:
# 미리 뽑아둔 CSV 파일에서 가져오기 // 위에서 보듯 P이상 자료
bot_count = pd.read_csv('bot_count.csv', index_col=0)
spt_count = pd.read_csv('spt_count.csv', index_col=0)
duo_count = pd.read_csv('duo_count.csv', index_col=0)

In [ ]:
bot_count

In [ ]:
# 각 챔피언 픽률 0.5% 이상인 챔피언만 남기기
bot_count = bot_count[bot_count['count'] >= 0.005*sum(bot_count['count'])/2]
spt_count = spt_count[spt_count['count'] >= 0.005*sum(spt_count['count'])/2]

In [ ]:
# DUO 매치업 50게임 이상인 조합만 남기기
duo_count = duo_count[duo_count['count'] >= 50]

In [ ]:
duo_count

,championId_x,championId_y,WR,count
399,15,3,0.528571,70.0
406,15,12,0.604278,187.0
408,15,16,0.507538,199.0
415,15,25,0.463115,244.0
416,15,26,0.531792,173.0
...,...,...,...,...
7148,711,350,0.533333,60.0
7190,777,53,0.454545,55.0
7236,777,350,0.430769,130.0
7284,875,235,0.540984,183.0


In [ ]:
# DataFrame 병합
duo_count = duo_count.merge(bot_count,
                            how = 'inner',
                            left_on = 'championId_x',
                            right_on = 'championId',
                            suffixes = ('', '_BOT')
                           ).\
                      merge(spt_count,
                            how = 'inner',
                            left_on = 'championId_y',
                            right_on = 'championId',
                            suffixes = ('', '_SPT'))

In [ ]:
duo_count = duo_count[['championId_x','championId_y','WR','count','WR_BOT','count_BOT','WR_SPT','count_SPT']]

In [ ]:
duo_count
# WR_BOT 이 WR보다 크면 조합 안좋겠지

In [ ]:
duo_count.sort_values('WR',ascending=False)

,championId_x,championId_y,WR,count,WR_BOT,count_BOT,WR_SPT,count_SPT
805,50,526,0.659574,94.0,0.555117,4563.0,0.512768,15194.0
27,50,12,0.648438,128.0,0.555117,4563.0,0.504591,39751.0
633,96,497,0.636364,55.0,0.504314,6258.0,0.506280,27627.0
696,50,44,0.631579,57.0,0.555117,4563.0,0.507701,5519.0
5,50,3,0.629630,54.0,0.555117,4563.0,0.489651,13818.0
...,...,...,...,...,...,...,...,...
91,18,26,0.352459,122.0,0.490314,19977.0,0.514815,15322.0
153,429,35,0.339286,56.0,0.472913,12423.0,0.491321,14691.0
765,222,79,0.333333,81.0,0.504149,38322.0,0.478861,5582.0
531,429,235,0.333333,78.0,0.472913,12423.0,0.490388,25125.0


In [ ]:
duo_count['score_SPT'] = norm.cdf((duo_count['WR']-duo_count['WR_BOT'])/
                                  np.sqrt((duo_count['WR']*(1-duo_count['WR']))/duo_count['count']+
                                          (duo_count['WR_BOT']*(1-duo_count['WR_BOT']))/duo_count['count_BOT']))
duo_count['score_BOT'] = norm.cdf((duo_count['WR']-duo_count['WR_SPT'])/
                                  np.sqrt((duo_count['WR']*(1-duo_count['WR']))/duo_count['count']+
                                          (duo_count['WR_SPT']*(1-duo_count['WR_SPT']))/duo_count['count_SPT']))
duo_count['score'] = norm.cdf((duo_count['WR']-0.5)/
                              np.sqrt((duo_count['WR']*(1-duo_count['WR']))/duo_count['count']))
# scor_SPT(BOT) 추가해서 유의한지 비교

In [ ]:
duo_count.sort_values('score',ascending=False)

,championId_x,championId_y,WR,count,WR_BOT,count_BOT,WR_SPT,count_SPT,score_SPT,score_BOT,score
261,21,57,0.564077,6063.0,0.520475,194680.0,0.531802,38221.0,1.000000e+00,9.999987e-01,1.000000e+00
114,21,32,0.544805,18469.0,0.520475,194680.0,0.521745,78364.0,1.000000e+00,1.000000e+00,1.000000e+00
238,21,53,0.553326,10314.0,0.520475,194680.0,0.523520,86096.0,1.000000e+00,1.000000e+00,1.000000e+00
324,21,89,0.538076,13565.0,0.520475,194680.0,0.512418,83065.0,9.999648e-01,1.000000e+00,1.000000e+00
243,67,53,0.546699,5332.0,0.518718,89405.0,0.523520,86096.0,9.999664e-01,9.995143e-01,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
683,145,875,0.436957,1380.0,0.484368,111601.0,0.452719,13790.0,2.087186e-04,1.302500e-01,1.169955e-06
86,360,25,0.419283,892.0,0.489920,89337.0,0.491850,36812.0,1.051232e-05,7.167398e-06,5.156797e-07
599,360,412,0.465795,5584.0,0.489920,89337.0,0.496203,106928.0,2.277409e-04,4.493381e-06,1.495591e-07
579,523,350,0.427301,2249.0,0.469932,52065.0,0.487878,95530.0,3.169083e-05,4.769564e-09,1.591937e-12


In [ ]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [ ]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [ ]:
champion_df

In [ ]:
# 챔피언 key -> name
duo_count = duo_count.merge(champion_df,
                            how = 'inner',
                            left_on = 'championId_x',
                            right_on = 'key').\
                      merge(champion_df,
                            how = 'inner',
                            left_on = 'championId_y',
                            right_on = 'key',
                            suffixes = ('_BOT','_SPT')
                           )

In [ ]:
# 필요한 컬럼만 취합
result = duo_count[['name_BOT','name_SPT','WR','count','score','score_BOT','score_SPT']].\
sort_values('score',ascending=False)

In [ ]:
result

,name_BOT,name_SPT,WR,count,score,score_BOT,score_SPT
261,미스 포츈,마오카이,0.564077,6063.0,1.000000e+00,9.999987e-01,1.000000e+00
324,미스 포츈,레오나,0.538076,13565.0,1.000000e+00,1.000000e+00,9.999648e-01
238,미스 포츈,블리츠크랭크,0.553326,10314.0,1.000000e+00,1.000000e+00,1.000000e+00
114,미스 포츈,아무무,0.544805,18469.0,1.000000e+00,1.000000e+00,1.000000e+00
243,베인,블리츠크랭크,0.546699,5332.0,1.000000e+00,9.995143e-01,9.999664e-01
...,...,...,...,...,...,...,...
683,카이사,세트,0.436957,1380.0,1.169955e-06,1.302500e-01,2.087186e-04
85,사미라,모르가나,0.419283,892.0,5.156797e-07,7.167398e-06,1.051232e-05
598,사미라,쓰레쉬,0.465795,5584.0,1.495591e-07,4.493381e-06,2.277409e-04
578,아펠리오스,유미,0.427301,2249.0,1.591937e-12,4.769564e-09,3.169083e-05


In [ ]:
result[result.name_BOT == '미스 포츈']

In [ ]:
result[result.name_SPT == '벨코즈']

,name_BOT,name_SPT,WR,count,score,score_BOT,score_SPT
476,케이틀린,벨코즈,0.583333,180.0,0.988329,0.991792,0.982990
486,루시안,벨코즈,0.533708,356.0,0.898827,0.928473,0.919091
477,베인,벨코즈,0.528662,314.0,0.845536,0.885494,0.637725
479,제라스,벨코즈,0.529801,151.0,0.768438,0.808126,0.725505
473,시비르,벨코즈,0.538462,78.0,0.752186,0.783017,0.720465
484,진,벨코즈,0.510857,875.0,0.739716,0.826341,0.753680
475,애쉬,벨코즈,0.518272,301.0,0.737107,0.795027,0.623959
482,드레이븐,벨코즈,0.531250,96.0,0.730251,0.765905,0.716132
481,직스,벨코즈,0.518293,164.0,0.680406,0.730463,0.392323
483,카이사,벨코즈,0.496855,318.0,0.455350,0.538834,0.671740
